# Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/')
%cd /content/drive/My\ Drive/

Mounted at /content/drive
/content/drive/My Drive


# Load Data

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import numpy as np
import sys
from tqdm import tqdm

In [3]:
from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128
default_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform = default_transform)

split_ratio = 0.8
total_size = len(trainset)
train_size = int(split_ratio * total_size)
valid_size = total_size - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

second_split = 0.05
total_size = len(train_dataset)
train_size = int(second_split * total_size)
discard_size = total_size - train_size
print(train_size)
train_dataset, discard_dataset = torch.utils.data.random_split(train_dataset, [train_size, discard_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform = default_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class_distribution = defaultdict(int)
for _, target in trainloader.dataset:
    class_name = classes[target]
    class_distribution[class_name] += 1
for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} samples")

Files already downloaded and verified
2000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified
plane: 213 samples
car: 201 samples
bird: 188 samples
cat: 178 samples
dog: 204 samples
ship: 209 samples
truck: 203 samples
frog: 197 samples
horse: 187 samples
deer: 220 samples


# ResNet50

In [6]:
from torch.optim.lr_scheduler import StepLR

resnet50 = models.resnet50(pretrained=True)
num_classes = 10
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50 = resnet50.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=.01, momentum=0.9)

scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [7]:
num_epochs = 40

for epoch in range(num_epochs):
  resnet50.train()
  running_loss = 0.0

  for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  scheduler.step()

  print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader)}')

resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = resnet50(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')

Epoch 1/40, Loss: 2.0065563395619392
Epoch 2/40, Loss: 1.1170721091330051
Epoch 3/40, Loss: 0.6263129413127899
Epoch 4/40, Loss: 0.2711408669129014
Epoch 5/40, Loss: 0.1954415359068662
Epoch 6/40, Loss: 0.1560950861312449
Epoch 7/40, Loss: 0.14892228902317584
Epoch 8/40, Loss: 0.1157290821429342
Epoch 9/40, Loss: 0.10193498036824167
Epoch 10/40, Loss: 0.06987422832753509
Epoch 11/40, Loss: 0.0437529182236176
Epoch 12/40, Loss: 0.0360456238995539
Epoch 13/40, Loss: 0.03671287698671222
Epoch 14/40, Loss: 0.03363746250397526
Epoch 15/40, Loss: 0.02452214391087182
Epoch 16/40, Loss: 0.03053745799115859
Epoch 17/40, Loss: 0.022522280749399215
Epoch 18/40, Loss: 0.0351531306077959
Epoch 19/40, Loss: 0.02789041325740982
Epoch 20/40, Loss: 0.016450900031486526
Epoch 21/40, Loss: 0.010624703900248278
Epoch 22/40, Loss: 0.005361257542972453
Epoch 23/40, Loss: 0.0033784229526645504
Epoch 24/40, Loss: 0.005346251233277144
Epoch 25/40, Loss: 0.003404341510758968
Epoch 26/40, Loss: 0.002944309566373

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import models

learning_rates = [1e-2, 1e-1]
num_epochs_list = [40, 50, 60]

best_accuracy = 0.0
best_params = {}
best_model = None

for lr in learning_rates:
    for num_epochs in num_epochs_list:

        resnet50 = models.resnet50(pretrained=True)
        num_classes = 10
        resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
        resnet50 = resnet50.to(device)


        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=0.9)
        scheduler = StepLR(optimizer, step_size=20, gamma=0.1)


        for epoch in range(num_epochs):
            resnet50.train()
            running_loss = 0.0

            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            scheduler.step()


        resnet50.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = resnet50(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'lr': lr, 'num_epochs': num_epochs}
            best_model = resnet50

print(f'Best Hyperparameters: {best_params}')
print(f'Best Valid Accuracy: {100 * best_accuracy:.2f}%')

correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = best_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the wo

Best Hyperparameters: {'lr': 0.01, 'num_epochs': 50}
Best Valid Accuracy: 65.94%
Test Accuracy: 65.79%


# Save Model

In [ ]:
torch.save(resnet50.state_dict(), 'resnet.pth')

# Load Model

In [ ]:
state_dict = torch.load('resnet.pth', map_location=torch.device(device))

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50.to(device)

resnet50.load_state_dict(state_dict)

<All keys matched successfully>